In [1]:
%AddJar https://repo1.maven.org/maven2/io/delta/delta-core_2.11/0.4.0/delta-core_2.11-0.4.0.jar

Starting download from https://repo1.maven.org/maven2/io/delta/delta-core_2.11/0.4.0/delta-core_2.11-0.4.0.jar
Finished download of delta-core_2.11-0.4.0.jar


In [2]:
import org.apache.spark.sql.delta.DeltaLog
import io.delta.tables._
import org.apache.spark.sql.functions._ 
import org.apache.spark.sql.{SaveMode, SparkSession, DataFrame}

# Create table 

In [3]:
  def createTable(data: DataFrame, tableName: String ): Unit = {
    data
      .write
      .format("delta")
      .mode(SaveMode.Overwrite)
      .save("/opt/deltalake/" + tableName)
  }

createTable: (data: org.apache.spark.sql.DataFrame, tableName: String)Unit


# Read table

In [4]:
 def readTable(tableName: String): DataFrame = {
   val df = spark
      .read
      .format("delta")
      .load("/opt/deltalake/" + tableName)
     df
  }

readTable: (tableName: String)org.apache.spark.sql.DataFrame


In [6]:
val data = readTable("sales")
data.show()

+---+----------+-------------------+-----+----------+
| id|product_id|         created_at|units|      date|
+---+----------+-------------------+-----+----------+
|  1|       205|1970-01-01 00:00:15|    1|1970-01-01|
|  2|       338|1970-01-01 00:01:15|    2|1970-01-01|
|  3|       461|1970-01-01 00:01:50|    5|1970-01-01|
|  4|       705|1970-01-06 03:25:55|    2|1970-01-06|
|  5|       919|1970-01-06 03:26:15|    2|1970-01-06|
|  6|       216|1970-01-06 03:26:35|    2|1970-01-06|
|  7|       668|2009-11-14 06:41:05|    5|2009-11-14|
|  8|       705|2009-11-14 06:41:10|    2|2009-11-14|
|  9|       900|2009-11-14 06:41:15|    5|2009-11-14|
| 10|       275|2009-11-14 06:41:20|    1|2009-11-14|
| 11|        80|2009-11-14 06:42:10|    1|2009-11-14|
| 12|        88|2010-03-26 21:46:45|    1|2010-03-26|
| 13|       240|2010-03-26 21:47:00|    5|2010-03-26|
| 14|       226|2010-03-26 21:47:20|    5|2010-03-26|
| 15|       463|2010-03-26 21:48:00|    3|2010-03-26|
| 16|      1014|2010-03-26 2

data = [id: string, product_id: string ... 3 more fields]


[id: string, product_id: string ... 3 more fields]

In [21]:
val deltaTable = DeltaTable.forPath(spark, "/opt/deltalake/sales")
deltaTable.delete(condition = "date = '1970-01-01'")  

deltaTable = io.delta.tables.DeltaTable@50480bf


io.delta.tables.DeltaTable@50480bf

In [38]:
deltaTable.updateExpr("date = '1970-01-06'", Map("created_at" -> "'ara'") ) 

In [39]:
readTable("sales").show()

+---+----------+-------------------+-----+----------+
| id|product_id|         created_at|units|      date|
+---+----------+-------------------+-----+----------+
|  4|       705|                ara|    2|1970-01-06|
|  5|       919|                ara|    2|1970-01-06|
|  6|       216|                ara|    2|1970-01-06|
|  7|       668|2009-11-14 06:41:05|    5|2009-11-14|
|  8|       705|2009-11-14 06:41:10|    2|2009-11-14|
|  9|       900|2009-11-14 06:41:15|    5|2009-11-14|
| 10|       275|2009-11-14 06:41:20|    1|2009-11-14|
| 11|        80|2009-11-14 06:42:10|    1|2009-11-14|
| 12|        88|2010-03-26 21:46:45|    1|2010-03-26|
| 13|       240|2010-03-26 21:47:00|    5|2010-03-26|
| 14|       226|2010-03-26 21:47:20|    5|2010-03-26|
| 15|       463|2010-03-26 21:48:00|    3|2010-03-26|
| 16|      1014|2010-03-26 21:48:30|    4|2010-03-26|
| 17|       900|2010-03-27 20:20:55|    1|2010-03-27|
| 18|       283|2010-03-27 20:21:30|    1|2010-03-27|
| 19|        42|2010-03-27 2

In [52]:
deltaTable.update(
  condition = expr("date = '1970-01-06'"),
  set = Map("created_at" -> expr("'raja'")))

In [53]:
readTable("sales").show()

+---+----------+-------------------+-----+----------+
| id|product_id|         created_at|units|      date|
+---+----------+-------------------+-----+----------+
|  4|       705|               raja|    2|1970-01-06|
|  5|       919|               raja|    2|1970-01-06|
|  6|       216|               raja|    2|1970-01-06|
|  7|       668|2009-11-14 06:41:05|    5|2009-11-14|
|  8|       705|2009-11-14 06:41:10|    2|2009-11-14|
|  9|       900|2009-11-14 06:41:15|    5|2009-11-14|
| 10|       275|2009-11-14 06:41:20|    1|2009-11-14|
| 11|        80|2009-11-14 06:42:10|    1|2009-11-14|
| 12|        88|2010-03-26 21:46:45|    1|2010-03-26|
| 13|       240|2010-03-26 21:47:00|    5|2010-03-26|
| 14|       226|2010-03-26 21:47:20|    5|2010-03-26|
| 15|       463|2010-03-26 21:48:00|    3|2010-03-26|
| 16|      1014|2010-03-26 21:48:30|    4|2010-03-26|
| 17|       900|2010-03-27 20:20:55|    1|2010-03-27|
| 18|       283|2010-03-27 20:21:30|    1|2010-03-27|
| 19|        42|2010-03-27 2

In [36]:
val newData = spark.range(500, 503).toDF

    deltaTable.as("oldData")
      .merge(
        newData.as("newData"),
        "oldData.id = newData.id")
      .whenMatched
      .update(Map("id" -> col("newData.id")))
      .whenNotMatched
      .insert(Map("id" -> col("newData.id")))
      .execute()

newData = [id: bigint]


[id: bigint]

In [38]:
readTable("numbers").show()

+---+
| id|
+---+
|501|
|500|
|  3|
|  1|
|502|
+---+

